In [1]:
require 'stopwords'
require 'rblearn'
require 'stemmify'
require 'numo/narray'
require 'rblearn'

false

# 70. データの入手・整形

文に関する極性分析の正解データを用い，以下の要領で正解データ（sentiment.txt）を作成せよ．

rt-polarity.posの各行の先頭に"+1 "という文字列を追加する（極性ラベル"+1"とスペースに続けて肯定的な文の内容が続く）
rt-polarity.negの各行の先頭に"-1 "という文字列を追加する（極性ラベル"-1"とスペースに続けて否定的な文の内容が続く）
上述1と2の内容を結合（concatenate）し，行をランダムに並び替える
sentiment.txtを作成したら，正例（肯定的な文）の数と負例（否定的な文）の数を確認せよ．

In [2]:
def knock70
  pos = File.open("../data/rt-polaritydata/rt-polarity.pos").read
  neg = File.open("../data/rt-polaritydata/rt-polarity.pos").read

  pos_tagged = pos.split("\n").map {|line| "+1 #{line}"}
  neg_tagged = neg.split("\n").map {|line| "-1 #{line}"}
  sen = (pos_tagged + neg_tagged).shuffle
  counter = {"+1" => 0, "-1" => 0}

  out = File.open("../data/sentiment.txt", "w")
  sen.each do |line|
    out.puts line
  end
  out.close

  open('../data/sentiment.txt').each_line do |line|
    counter[line[0..1]] += 1
  end

  return counter
end

:knock70

In [3]:
knock70

{"+1"=>5330, "-1"=>5330}

# 71. ストップワード
英語のストップワードのリスト（ストップリスト）を適当に作成せよ．さらに，引数に与えられた単語（文字列）がストップリストに含まれている場合は真，それ以外は偽を返す関数を実装せよ．さらに，その関数に対するテストを記述せよ．

# 72. 素性抽出
極性分析に有用そうな素性を各自で設計し，学習データから素性を抽出せよ．素性としては，レビューからストップワードを除去し，各単語をステミング処理したものが最低限のベースラインとなるであろう．

In [4]:
def knock72
  labels = []
  features = []

  File.foreach('../data/sentiment.txt') do |line|
    line.chomp!
    label = line[0..1]
    feature = line[3..-1]
    labels << (label == '+1' ? 1 : 0)
    features << feature
  end

  cv = Rblearn::CountVectorizer.new(lambda{|feature| feature.split.map(&:stem).map(&:downcase)}, 1, 0.7)
  cv.fit_transform(features)

  y = Numo::Int8.zeros([labels.size, 1])
  labels.each_with_index do |label, i|
    y[i, 0] = label
  end

  return [cv, y]
end

:knock72

In [5]:
cv, y = knock72

1

1

# 73. 学習

72で抽出した素性を用いて，ロジスティック回帰モデルを学習せよ．

In [28]:
class LogisticRegressionModel
  attr_accessor :w

  def initialize(eta, threshold)
    @eta = eta
    @threshold = threshold
  end
  
  
  def h(x)
    z = x.dot(@w)
    return 1 / (1 + Numo::NMath.exp(-z))
  end


  def fit(x, y)
    @w = Numo::Float64.new([x.shape[1], 1]).rand
    5.times do |t|

      # p y-h(x)
      # NOTE: there is something bug
      @w = @w - @eta * x.transpose.dup.dot(h(x) - y) # x.T.dot(h(x) - y): gradient
      puts "#iter: #{t}, accuracy: #{score(x, y)}" if t % 5 == 0
    end
  end


  def predict(x)
    probabilities = h(x)
    return probabilities > @threshold
  end


  def predict_prob(x)
    return h(x)
  end


  def score(x, y)
    predicted_class = predict(x)
    num_data = predicted_class.shape[0]

    py = predicted_class.to_a
    ry = y.to_a

    num_correct = 0
    num_data.times do |t|
      num_correct += 1 if py[t] == ry[t]
    end
    return num_correct.to_f / num_data
  end
end

:score

In [7]:
x = cv.doc_matrix
x_train, y_train, x_test, y_test = Rblearn::CrossValidation.train_test_split(x, y, 0.5).map(&:dup)

model = LogisticRegressionModel.new(0.02, 0.7)
model.fit(x_train, y_train)

puts "test accuracy: #{model.score(x_test, y_test)}"

#iter: 0, accuracy: 0.500187617260788
#iter: 5, accuracy: 0.5581613508442776
#iter: 10, accuracy: 0.5864915572232645
#iter: 15, accuracy: 0.6054409005628518
#iter: 20, accuracy: 0.6210131332082551
#iter: 25, accuracy: 0.6330206378986867
test accuracy: 0.3600375234521576


# 74. 予測

73で学習したロジスティック回帰モデルを用い，与えられた文の極性ラベル（正例なら"+1"，負例なら"-1"）と，その予測確率を計算するプログラムを実装せよ．

In [34]:
model.predict(x_test).each_with_index do |arr, index|
  puts arr
  break if index == 30
end

0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
1
0
0
1
1
0


# 75. 素性の重み
73で学習したロジスティック回帰モデルの中で，重みの高い素性トップ10と，重みの低い素性トップ10を確認せよ．

In [14]:
feature_names = cv.get_feature_names
weights = model.w.to_a.map{|arr| arr[0]}
name_weights = {}

feature_names.size.times do |t|
  name_weights[feature_names[t]] = weights[t]
end

puts "high"
name_weights.
  sort{|(_, v1), (_, v2)| v2 <=> v1}.
  first(10).
  map{|arr| arr.join("\t")}.
  each do |record|
    puts record
  end

puts "low"
name_weights.
  sort{|(_, v1), (_, v2)| v1 <=> v2}.
  first(10).
  map{|arr| arr.join("\t")}.
  each do |record|
    puts record
  end

1

high
tragedi	5.533292656509563
distanc	5.378454726024288
kick	5.101951269716
rel	5.066501398820613
stuart	5.0333344181269775
stereotyp	4.886598383118593
stage	4.728879366486668
teacher	4.572382275855271
wish	4.541008714905942
skin	4.522752797367518
low
transform	-5.676555627079279
twice	-4.180186273969201
contemporari	-4.119185328894374
unlik	-4.050743764233494
hand	-3.7945183277249255
succumb	-3.7264854000511427
obsess	-3.6547692665495615
inner	-3.6260473664174806
reminisc	-3.592334680394191
chemistri	-3.4140663314468775


1

# 76. ラベル付け

学習データに対してロジスティック回帰モデルを適用し，正解のラベル，予測されたラベル，予測確率をタブ区切り形式で出力せよ．

In [24]:
a = y_test.to_a.map{|arr| arr[0]}
b = model.predict(x_test).to_a.map{|arr| arr[0]}
c = model.predict_prob(x_test).to_a.map{|arr| arr[0]}

output = open('../data/knock76.tsv', 'w')

a.size.times do |t|
  puts "#{a[t]}\t#{b[t]}\t#{c[t]}" if t < 30
  output.puts "#{a[t]}\t#{b[t]}\t#{c[t]}"
end

output.close

1	1	0.9999691570177368
1	1	0.9990848192291869
1	1	0.9871991121228477
1	1	0.8445673254075029
0	1	0.5605188137447192
0	0	0.49714818223415636
0	1	0.9999986746051605
1	1	0.9997669959821033
1	1	0.9998234493131791
0	1	0.7941341961964655
1	1	0.949900664390127
1	1	0.9998197454641644
0	1	0.946746589087656
1	0	0.43213023953295665
0	1	0.9997769010500968
0	1	0.9989366012614198
1	1	0.9263939475522653
1	0	0.29042444666376477
0	1	0.9998415776384834
0	1	0.9874574895172559
0	1	0.9985453105630353
0	1	0.7993134501261226
1	1	0.8047494763801611
1	1	0.9998318617956068
0	1	0.996423503007124
0	1	0.9999998011745099
1	0	0.23610523815499684
1	1	0.9652479357643858
0	1	0.9434602299173194
1	1	0.9882528790542259


# 77. 正解率の計測

76の出力を受け取り，予測の正解率，正例に関する適合率，再現率，F1スコアを求めるプログラムを作成せよ．

In [26]:
num_all = 0
num1 = 0.0
num2 = 0.0
num3 = 0.0

# a is the actual label
# b is the predicted label
File.foreach('../data/knock76.tsv') do |line|
  num_all += 1

  a, b, _ = line.split.map(&:to_f)
  
  if b == 1
    if a == 1
      num1 += 1
    elsif a == 0
      num2 += 1
    end
  elsif b == 0 && a == 1
    num3 += 1
  end
end

precision = num1 / (num1 + num2)
recall = num3 / (num1 + num3)

puts "precision: #{precision}"
puts "recall: #{recall}"
puts "F-1 score: #{2*precision*recall / (precision + recall)}"

precision: 0.4161551681807022
recall: 0.37546057479734707
F-1 score: 0.3947618779338646
